# Implementação do Algoritmo Simplex

### Importação de bibliotecas

In [6]:
import math
import numpy as np
import matplotlib.pyplot as plt

### Inicialização do problema

In [16]:
# Definição do problema de programação linear
# coeficientes da função objetivo
c = [1, 1, 0, 0, 0]

# matriz de coeficientes das restrições
A = [
    [-1, 1, 1, 0, 0],
    [ 1, 0, 0, 1, 0],
    [ 0, 1, 0, 0, 1]
]

# vetor de termos independentes das restrições
b = [2, 4, 4]



def para_tableau(c, A, b):
    # Combina cada equação (linha) de A com seu valor correspondente em b
    # Isso cria as linhas iniciais do tableau (excluindo a linha da função objetivo)
    xb = []
    for equation, right_hand_side in zip(A, b):
        tableau_row = equation + [right_hand_side]
        print(tableau_row)
        xb.append(tableau_row)
    # Cria a linha da função objetivo para o tableau
    # Adiciona um zero no final para representar o valor inicial da função objetivo
    z = c + [0]

    # Retorna o tableau completo
    # Combina as linhas das restrições (xb) com a linha da função objetivo (z)
    return xb + [z]

def pode_ser_melhorado(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])

def obter_posicao_pivo(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    
    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column

def passo_pivo(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
   
    return new_tableau

def eh_basica(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def obter_solucao(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns:
        solution = 0
        if eh_basica(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)
        
    return solutions

def simplex(c, A, b):
    tableau = para_tableau(c, A, b)

    while pode_ser_melhorado(tableau):
        pivot_position = obter_posicao_pivo(tableau)
        tableau = passo_pivo(tableau, pivot_position)

    return obter_solucao(tableau)

solucao = simplex(c, A, b)
print('solução: ', solucao)


[-1, 1, 1, 0, 0, 2]
[1, 0, 0, 1, 0, 4]
[0, 1, 0, 0, 1, 4]
-------------------------
[1, 1, 0, 0, 0, 0]
-------------------------
solution:  [np.float64(4.0), np.float64(4.0), np.float64(2.0), 0, 0, 0]
